# Library

In [2]:
import pandas as pd
import numpy as np
import re
import warnings

from konlpy.tag import Mecab
from gensim.models import FastText
from nltk.tokenize import sent_tokenize
from sqlalchemy import create_engine
from tqdm import tqdm_notebook

In [3]:
import pandas as pd
import numpy as np
import pickle as pk
import re
import tensorflow as tf

from gensim.models import FastText
from sentence_transformers import SentenceTransformer
from sqlalchemy import create_engine
from nltk import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
# 경고무시를 위한 코드
warnings.filterwarnings('ignore')

# 데이터 로드

In [4]:
case_list_사기 = pd.read_csv('./00. Data/case_list_사기.csv')

## 문장 토큰화
- 각 판례 본문을 문장으로 토큰화

In [37]:
# 각 판례의 모든 문장을 리스트에 저장
sentence_tokened = []

for x in tqdm_notebook(case_list_사기['X']):
    sentence_tokened.extend(sent_tokenize(x))
# append와 extend의 차이: append는 x 그 자체를 원소로 넣고 extend는 iterable의 각 항목들을 넣는다

  0%|          | 0/2262 [00:00<?, ?it/s]

## 형태소 토큰화 전 전처리

In [39]:
clean_sentence_tokened = []
for sent in tqdm_notebook(sentence_tokened):
    clean_sentence_tokened.append(re.sub('[^가-힣]', '', sent))

  0%|          | 0/96981 [00:00<?, ?it/s]

## 형태소 토큰화

- mecab-ko-dic은 오픈 소스 형태소 분석 엔진인 MeCab을 사용하여, 한국어 형태소 분석을 하기 위한 프로젝트
- 현재 KoNLPy는 윈도우에서 Kkma(), Komoran()만 지원합니다. Mecab(), Hannanum() 는 지원되지 않습니다.

In [ ]:
from konlpy.tag import Okt
okt = Okt()

tokenized_vocab = []

for sent in tqdm_notebook(clean_sentence_tokened):
    token_word = okt.morphs(sent)
    tokenized_vocab.append(token_word)


# 학습 - FastText

In [ ]:
model = FastText(tokenized_vocab, vector_size=512, window=7, min_count=5, workers=4)

In [35]:
model.save('fasttext_okt.model')

# Embedding

In [19]:
# 모델 로드
model = FastText.load('./00. Data/fasttext_okt.model')

### 문장 토큰화 전 전처리

In [42]:
sentence_tokened = []

for x in tqdm_notebook(case_list_사기['X']):
    sentence_tokened.append(sent_tokenize(x))

  0%|          | 0/2262 [00:00<?, ?it/s]

In [21]:
embeddings = []

for sentences in tqdm_notebook(sentence_tokened):
    embedding = None
    count = 0
    for sentence in sentences:
        if embedding is None:
            embedding = model.wv.get_sentence_vector(sentence)
        else:
            embedding += model.wv.get_sentence_vector(sentence)
        count += 1
    embeddings.append(embedding/count)

  0%|          | 0/2262 [00:00<?, ?it/s]

In [31]:
with open('./00. Data/fasttext_embedding.pickle', 'wb') as fw:
    pk.dump(embeddings, fw)

# Fasttext 성능 테스트

In [32]:
# 모델, 임베딩 로드
fasttext = FastText.load('./00. Data/fasttext_okt.model')
with open('./00. Data/fasttext_embedding.pickle', 'rb') as fr:
    fasttext_embedding = pk.load(fr)

### 사용자 입력 전처리

In [29]:
from hanja_hangul.src import hanjahangul
# 사용자 입력 전처리
user_input = "저는 3년 전부터 甲에게 그 남편의 사업자금 명목으로 수 차례에 걸쳐 1,500만원을 빌려주었습니다. \
    甲은 돈을 빌릴 당시에는 남편의 건축사업이 잘되면 이자는 물론 아파트분양까지 책임지겠다고 하여 믿고 빌려주었는데, \
    이제 와서는 건축경기가 좋지 않아 파산위기에 처해 있으니 마음대로 하라고 합니다. 이 경우 甲을 사기죄로 처벌할 수 있는지요?"

user_input = hanjahangul.hanja_to_hangul_dueum(user_input)
clear_user_input = re.sub("[^가-힣. ]", '', user_input)
clear_user_input

'저는 년 전부터 갑에게 그 남편의 사업자금 명목으로 수 차례에 걸쳐 만원을 빌려주었습니다.     갑은 돈을 빌릴 당시에는 남편의 건축사업이 잘되면 이자는 물론 아파트분양까지 책임지겠다고 하여 믿고 빌려주었는데     이제 와서는 건축경기가 좋지 않아 파산위기에 처해 있으니 마음대로 하라고 합니다. 이 경우 갑을 사기죄로 처벌할 수 있는지요'

### 사용자 입력 문장 토큰화

In [30]:
# 사용자 입력 문장 토큰화
user_sentence_tokened = sent_tokenize(clear_user_input)
user_sentence_tokened

['저는 년 전부터 갑에게 그 남편의 사업자금 명목으로 수 차례에 걸쳐 만원을 빌려주었습니다.',
 '갑은 돈을 빌릴 당시에는 남편의 건축사업이 잘되면 이자는 물론 아파트분양까지 책임지겠다고 하여 믿고 빌려주었는데     이제 와서는 건축경기가 좋지 않아 파산위기에 처해 있으니 마음대로 하라고 합니다.',
 '이 경우 갑을 사기죄로 처벌할 수 있는지요']

### 사용자 입력 임베딩

In [31]:
# 사용자 입력 임베딩
embedding = None
count = 0
for sentence in user_sentence_tokened:
    if embedding is None:
        embedding = fasttext.wv.get_sentence_vector(sentence)
    else:
        embedding += fasttext.wv.get_sentence_vector(sentence)
    count += 1

f_embedding = embedding / count

### 코사인 유사도 계산

In [32]:
f_similarity = cosine_similarity([f_embedding], fasttext_embedding)
f_similarity = list(enumerate(f_similarity[0]))
f_similarity = sorted(f_similarity, key = lambda x: x[1], reverse=True)

### 유사 판례 (상위 5개) 확인

In [33]:
precidents = []

for i in range(5):
    idx = f_similarity[i][0]
    sim = round(f_similarity[i][1] * 100, 2)

    precidents.append([case_list_사기['판례일련번호'][idx], case_list_사기['사건명'][idx], case_list_사기['사건종류명'][idx], case_list_사기['판례내용'][idx], sim])

In [34]:
f_result = pd.DataFrame(precidents, columns=['판례일련번호', '사건명', '사건종류명', '판례내용', '유사도'])
f_result

,판례일련번호,사건명,사건종류명,판례내용,유사도
0,76900,사기피고사건,형사,"【피 고 인】 【주 문】 피고인은 무죄【이 유】 이 사건 공소사실은,...",96.12
1,192996,살인(예비적죄명:교통사고처리특례법위반)·사기,형사,"【피 고 인】 【항 소 인】 검사【검 사】 강화연(기소, 공판), 박병모(공판...",95.90
2,215295,"사기,횡령",형사,"【상고인, 피고인】 피고인【원심판결】 제1심 광주지방법원 목포지원, 제2심 광주고등...",95.80
3,104187,사기,형사,【피 고 인】 【상 고 인】 피고인【원심판결】 \n서울형사지방법원 1987.7....,95.75
4,71591,현주건조물방화·사기미수,형사,【피고인】 【항소인】 피고인【변호인】 한밭 법무법인 담당변호사 명을식【원심판결】...,95.72
